In [1]:
import pandas as pd

In [2]:
## Create a Basic DataFrame
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9],[10,11,12]], 
                  columns=["A", "B", "C"],
                  index=["x", "Y", "Z", "BA"])


In [3]:
## Some useful functions about dataframes

df.head() # Show the first 5 rows
# df.head(2) # first 2 rows
# df.tail(2) # last 2 rows
# df.columns # Show the column names
# df.index # Show the row names
# df.info() # Show info such as the columns and their types
# statsDf = df.describe() # Means, maxes, stats, etc
# display(statsDf)
# display(df.head())

# # Columns and unique Values
# A = df['A']
# uVals = A.unique()
# display(A)
# display(type(A))
# display(uVals)

df.shape # (rows,cols)
df.size # Total Number of elements


12

In [ ]:
## Read from a parquet
# results = pd.read_parquet('./data/results.parquet')
# results.to_parquet

In [ ]:
## Read excel
# olympics_data = pd.read_excel('./data/olympics-data.xlsx', sheet_name='results')
# olympics_data.to_excel()


In [95]:
## Loading from a CSV File
coffee = pd.read_csv('./warmup-data/coffee.csv')
# # coffee.to_parquet(path="./check.parquet") # Save it

In [ ]:
## Accessing Data 
# coffee.sample(3, random_state=2) # Get some random rows (repeatable with random_state)
coffee.loc[:, ["Day", "Units Sold"]] # Loc returns just the rows and cols that you want
coffee.iloc[:,1] # Allows you to use integer based indexing 
coffee.Day # Grab a single column
# coffee.index = coffee.Day # Set the index
# coffee


# Note that for single values you can use iat or at, for slightly better performance...

0        Monday
1        Monday
2       Tuesday
3       Tuesday
4     Wednesday
5     Wednesday
6      Thursday
7      Thursday
8        Friday
9        Friday
10     Saturday
11     Saturday
12       Sunday
13       Sunday
Name: Day, dtype: object

In [94]:
## Setting various bits of data
coffee.loc[0,"Units Sold"] # has value of 25
coffee.loc[0:2,'Units Sold'] = 20 #Now it has a value of 20
coffee.loc[0,'Units Sold']


np.int64(20)

In [ ]:
## More Useful functions
coffee.sort_values(["Coffee Type", "Units Sold"], ascending=[False, True])
coffee['Coffee Type'].unique()

# If you need to loop through
# Note that this is not very efficient.  Try to stick to built in pandas methods
for idx, row in coffee.iterrows():
    print(f"{idx} {row}\n")

In [ ]:
## Load more Data
bios = pd.read_csv('./data/bios.csv')

In [199]:
## Filtering Operations
rowPts = (bios['height_cm'] > 215) & (bios['born_country']=='USA')
cols_to_keep =  ["NOC", "name", "born_region","height_cm","born_country"]
filtered_df = bios.loc[rowPts, cols_to_keep]
filtered_df = bios[rowPts][cols_to_keep] # this is equivelent to the previos line
sorted_filtered_df = filtered_df.sort_values("height_cm", ascending=False)
sorted_filtered_df.head()



,NOC,name,born_region,height_cm,born_country
5781,United States,Tommy Burleson,North Carolina,223.0,USA
6722,United States,Shaquille O'Neal,New Jersey,216.0,USA
6937,United States,David Robinson,Florida,216.0,USA
123850,United States,Tyson Chandler,California,216.0,USA


In [ ]:
## Filter with string operations
name_pts = bios['name'].str.contains(r're[nw]', case=False)
country_pts = bios['born_country'].isin(['USA', 'FRA'])
notNanPts = ~bios['height_cm'].isna()
bios[name_pts & notNanPts & country_pts][cols_to_keep]

# Filter using complex Query 
# Would need to learn this way more...
bios.query('name == "Trena King"')

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
2200,2210,Trena King,1958-01-17,Kingwood,West Virginia,USA,United States,172.0,68.0,NaN


In [235]:
## Adding / Removing Columns
# Add Price Uniformly
coffee['Price'] = 4.99 # Set for all rows

import numpy as np
coffee['new_price'] = np.where(coffee['Coffee Type'] == 'Espresso', 3.99,5.99) #espresso is 399 and all else is 599



## Practice with Merging DFS
price_df = pd.DataFrame([['Espresso', 3.99], ['Latte',5.99]], columns=['Coffee Type', 'DF Price'])
right_df = coffee.merge(price_df, 'right')
left_df = coffee.merge(price_df, 'left')
coffee = left_df.copy() # This is how you make a deep copy
# display(right_df)
# display(left_df)

## Drop the initial columns
coffee = coffee.drop(columns=['Price', 'new_price'])

## Perform some simple calculations
coffee = coffee.rename(columns={'DF Price':'Price'})
coffee['revenue'] = coffee['Units Sold'] * coffee['Price']


In [ ]:
bios_new = bios.copy()
# bios_new
bios_new['first_name'] = bios_new['name'].str.split(' ').str[0]
# bios_new.info()
bios_new['born_year_str'] = bios_new['born_date'].str.split('-').str[0]
bios_new['born_datetime'] = pd.to_datetime(bios_new['born_date'],format="%Y-%m-%d")
bios_new['born_year_datetime'] = bios_new['born_datetime'].dt.year
check = bios_new[['born_date', 'born_datetime', 'born_year_str', 'born_year_datetime', 'born_year_datetime_str']]
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145500 entries, 0 to 145499
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   born_date               143693 non-null  object        
 1   born_datetime           143693 non-null  datetime64[ns]
 2   born_year_str           143693 non-null  object        
 3   born_year_datetime      143693 non-null  float64       
 4   born_year_datetime_str  145500 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 5.6+ MB


In [280]:
## Complicated Column using apply
bios_new['height_category'] = bios_new['height_cm'].apply(lambda x:'tall' if x > 165 else 'shorty')
sum(bios_new['height_category'] == 'tall') / bios_new.shape[0]

0.6190790378006873

In [284]:
## Apply even more complicated function defined
# THis is not as efficient as built ins, but sometimes that is ok
def getWeightClass(row):
    if row['weight_kg'] < 70 and row['height_category']=='shorty':
        return 'lightweight'
    elif row['weight_kg'] >= 80 or row['height_category'] == 'tall':
        return 'middleweight'
    else:
        return 'heavyweight'

bios_new['weight_category'] = bios_new.apply(getWeightClass, axis=1)
bios_new['weight_category']

0          heavyweight
1         middleweight
2         middleweight
3         middleweight
4          heavyweight
              ...     
145495    middleweight
145496    middleweight
145497     lightweight
145498    middleweight
145499     heavyweight
Name: weight_category, Length: 145500, dtype: object

In [ ]:
## Merge and Concatenate Data
# Let's convert country abr to full country
# Load the metadata file
nocs_df = pd.read_csv('./data/noc_regions.csv')

# nocs_df.head(20)
bios_new.head()
bios_new2 = bios_new.merge(nocs_df, how='left', right_on='NOC', left_on='born_country')
bios_new2.rename(columns={'region':'born_country_full'}, inplace=True)
nanPts = pd.isna(bios_new2['born_country_full'])
country_pts = bios_new2['NOC_x'] != bios_new2['born_country_full']
# npIdx = np.where(nanPts & country_pts) # you can also do this to get index instead of boolean array 
outsiders_df = bios_new2[(bios_new2['NOC_x'] != bios_new2['born_country_full']) & ~pd.isna(bios_new2['born_country_full'])]
outsiders_df.head()

fra_df = bios_new[bios_new['born_country'] == "FRA"]
grb_df = bios_new[bios_new['born_country'] == "GBR"]
comb_df = pd.concat([fra_df, grb_df])
comb_df.tail(2)

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date,first_name,born_year_str,born_datetime,born_year_datetime,born_year_datetime_str,height_category,weight_category
144659,148355,Bruce Mouat,1994-08-27,Edinburgh,Scotland,GBR,Great Britain,NaN,NaN,NaN,Bruce,1994,1994-08-27,1994.0,1994.0,shorty,heavyweight
144661,148357,Leonie Gerken Schofield,1998-02-15,Chelmsford,England,GBR,Great Britain,NaN,NaN,NaN,Leonie,1998,1998-02-15,1998.0,1998.0,shorty,heavyweight
144662,148358,Makayla Gerken Schofield,1999-06-04,Chelmsford,England,GBR,Great Britain,NaN,NaN,NaN,Makayla,1999,1999-06-04,1999.0,1999.0,shorty,heavyweight
144663,148359,Ollie Davies,1997-05-15,Guildford,England,GBR,Great Britain,NaN,NaN,NaN,Ollie,1997,1997-05-15,1997.0,1997.0,shorty,heavyweight
144664,148360,William Feneley,1999-07-13,Norwich,England,GBR,Great Britain,NaN,NaN,NaN,William,1999,1999-07-13,1999.0,1999.0,shorty,heavyweight
144666,148362,Natasha McKay,1995-01-14,Dundee,Scotland,GBR,Great Britain,NaN,NaN,NaN,Natasha,1995,1995-01-14,1995.0,1995.0,shorty,heavyweight
144667,148363,Lewis Gibson,1994-05-01,Prestwick,Scotland,GBR,Great Britain,NaN,NaN,NaN,Lewis,1994,1994-05-01,1994.0,1994.0,shorty,heavyweight
144668,148364,Katie Ormerod,1997-08-25,Bradford,England,GBR,Great Britain,NaN,NaN,NaN,Katie,1997,1997-08-25,1997.0,1997.0,shorty,heavyweight
144669,148365,Huw Nightingale,2001-11-12,Bolton,England,GBR,Great Britain,NaN,NaN,NaN,Huw,2001,2001-11-12,2001.0,2001.0,shorty,heavyweight
144670,148366,Brogan Crowley,1994-07-20,Saddleworth,England,GBR,Great Britain,NaN,NaN,NaN,Brogan,1994,1994-07-20,1994.0,1994.0,shorty,heavyweight


In [362]:
## Interpolate
coffee_new = coffee.copy()
coffee_new.loc[1:2,'Units Sold'] = np.nan
coffee_new['Units Sold Interpolate'] = coffee_new['Units Sold'].interpolate().round(1)
coffee_new2 = coffee_new.dropna(subset=['Units Sold'])
coffee_new2

,Day,Coffee Type,Units Sold,revenue,price,Price,Units Sold Interpolate
0,Monday,Espresso,25.0,99.75,4.99,3.99,25.0
3,Tuesday,Latte,20.0,119.80,4.99,5.99,20.0
4,Wednesday,Espresso,35.0,139.65,4.99,3.99,35.0
5,Wednesday,Latte,25.0,149.75,4.99,5.99,25.0
6,Thursday,Espresso,40.0,159.60,4.99,3.99,40.0
7,Thursday,Latte,30.0,179.70,4.99,5.99,30.0
8,Friday,Espresso,45.0,179.55,4.99,3.99,45.0
9,Friday,Latte,35.0,209.65,4.99,5.99,35.0
10,Saturday,Espresso,45.0,179.55,4.99,3.99,45.0
11,Saturday,Latte,35.0,209.65,4.99,5.99,35.0


In [396]:
## Data Aggregation
o = bios_new2['born_country_full'].value_counts()
meanCountryCount = o.mean
med = o.median()

coffee.groupby(['Coffee Type']).agg({'Units Sold':'mean', 'Price': 'median'})

# Pivot Tables
pivot = coffee.pivot_table(columns=['Coffee Type'], index='Day', values='revenue',aggfunc=['sum','mean'])
pivot


sum             mean        
Coffee Type Espresso   Latte Espresso   Latte
Day                                          
Friday        179.55  209.65   179.55  209.65
Monday         99.75   89.85    99.75   89.85
Saturday      179.55  209.65   179.55  209.65
Sunday        179.55  209.65   179.55  209.65
Thursday      159.60  179.70   159.60  179.70
Tuesday       119.70  119.80   119.70  119.80
Wednesday     139.65  149.75   139.65  149.75

In [422]:
## Advanced Functions
coffee_new2.loc[:,'yesterday_revenue'] = coffee_new2['revenue'].shift(2) # Moves the values by 2 rows (nan pad)
coffee_new2.loc[:,'percentChang'] = (coffee_new2['revenue'] / coffee_new2['yesterday_revenue'] * 100 - 100).round(1)

coffee_new2.loc[:,'revenue_rank'] = coffee_new2['revenue'].rank() # Rank == 1 is least revenue, rank == end == most revenue
# coffee_new2.sort_values('revenue_rank')

coffee_new2.loc[:,'CumulativeRevenue'] = coffee_new2['revenue'].cumsum()
coffee_new2.loc[:,'CumulativeMaxRevenue'] = coffee_new2['revenue'].cummax()
coffee_new2

,Day,Coffee Type,Units Sold,revenue,price,Price,Units Sold Interpolate,yesterday_revenue,percentChang,revenue_rank,CumulativeRevenue,CumulativeMaxRevenue
0,Monday,Espresso,25.0,99.75,4.99,3.99,25.0,NaN,NaN,1.0,99.75,99.75
3,Tuesday,Latte,20.0,119.80,4.99,5.99,20.0,NaN,NaN,2.0,219.55,119.80
4,Wednesday,Espresso,35.0,139.65,4.99,3.99,35.0,99.75,40.0,3.0,359.20,139.65
5,Wednesday,Latte,25.0,149.75,4.99,5.99,25.0,119.80,25.0,4.0,508.95,149.75
6,Thursday,Espresso,40.0,159.60,4.99,3.99,40.0,139.65,14.3,5.0,668.55,159.60
7,Thursday,Latte,30.0,179.70,4.99,5.99,30.0,149.75,20.0,9.0,848.25,179.70
8,Friday,Espresso,45.0,179.55,4.99,3.99,45.0,159.60,12.5,7.0,1027.80,179.70
9,Friday,Latte,35.0,209.65,4.99,5.99,35.0,179.70,16.7,11.0,1237.45,209.65
10,Saturday,Espresso,45.0,179.55,4.99,3.99,45.0,179.55,0.0,7.0,1417.00,209.65
11,Saturday,Latte,35.0,209.65,4.99,5.99,35.0,209.65,0.0,11.0,1626.65,209.65
